## Homework 8:

__Exercise 1.__ Go to prosite and find the Gamma-glutamyl phosphate reductase signature. Write a script to detect all yeast proteins that have the Gamma-glutamyl phosphate reductase signature. Your script should output a dataframe with the folowing info about the proteins: accession number, match to the first, span of the motif, and the protein's description.

**NOTE:** You can just copy the file with all the yeast protein form last class in the current directory, you don't need to download it again.

```
[VA]-x(5)-A-[LIVAMTCK]-x-[HWFY]-[IM]-x(2)-[HYWNRFT]-[GSNT]-[STAG]-x(0,1)-H-[ST]-[DE]-x(1,2)-I
```

In [5]:
protFile = 'orf_trans.fasta'

pattern = "[VA]\w\w\w\w\wA[LIVAMTCK]\w[HWFY][IM]\w\w[HYWNRFT][GSNT][STAG]\w?H[ST][DE]\w{1,2}I"

from Bio import SeqIO
import pandas as pd
import re

def findinseqfile(pattern,filein):
    information = []
    for seq_record in SeqIO.parse(filein, "fasta"):
        result = re.search(pattern,str(seq_record.seq))
        if result:
            information.append([seq_record.name ,result.group(), result.span(), str(seq_record.description)])

    return pd.DataFrame(information, columns=['acc','match','start_end','seq'])

findinseqfile(pattern, protFile)

,acc,match,start_end,seq
0,YOR323C,VTSTESAIQHINTHSSRHTDAI,"(338, 360)","YOR323C PRO2 SGDID:S000005850, Chr XV from 922..."


__Exercise 2.__ Now do the same for the Hexapeptide-repeat containing-transferases signature.

```
[LIV]-[GAED]-x(2)-[STAV]-x-[LIV]-x(3)-[LIVAC]-x-[LIV]-[GAED]-x(2)-[STAVR]-x-[LIV]-[GAED]-x(2)-[STAV]-x-[LIV]-x(3)-[LIV]
```

method1: create two dataframes that have either and merge inner
method2: search for the first one and search for the second one, write only if both are found


In [14]:
protFile = 'orf_trans.fasta'
newpattern = "[LIV][GAED]\w\w[STAV]\w[LIV]\w\w\w[LIVAC]\w[LIV][GAED]\w\w[STAVR]\w[LIV][GAED]\w\w[STAV]\w[LIV]\w\w\w[LIV]"

findinseqfile(newpattern, protFile)

,acc,match,start_end,seq
0,YDL055C,IDPTAKISSTAKIGPDVVIGPNVTIGDGV,"(256, 285)","YDL055C PSA1 SGDID:S000002213, Chr IV from 356..."
1,YJL218W,IGGGVSIIPGVNIGKNSVIAAGSVVIRDI,"(138, 167)","YJL218W YJL218W SGDID:S000003754, Chr X from 2..."


__Exercise 3.__ Now find the 14-3-3 proteins signatures. The 14-3-3 proteins seem to have multiple biological activities and play a key role in signal transduction pathways and the cell cycle. The prosite database uses two motifs to determine members of this family.

Write a script to search for proteins in yeast that have both domains in either order. You should find two proteins.

Your script should show a dataframe with the proteins: accession number, match to the first motif, span of the first motif, match to the second motif, span of the second motif, and the proteins description.

Although your regex doesn't need to match the domains in the reverse order for it to identify both yeast proteins, I would like for you to write a regex that would be able to identify such a case for this exercise purpose.

```
[RA]-N-L-[LIV]-S-[VG]-[GA]-Y-[KN]-N-[IVA]
```

and

```
Y-K-[DE]-[SG]-T-L-I-[IML]-Q-L-[LF]-[RHC]-D-N-[LF]-T-[LS]-W-[TANS]-[SAD]
```

In [17]:
protFile = 'orf_trans.fasta'
pattern1 = "[RA]NL[LIV]S[VG][GA]Y[KN]N[IVA]"
pattern2 = "YK[DE][SG]TLI[IML]QL[LF][RHC]DN[LF]T[LS]W[TANS][SAD]"

def intersect(pattern1, pattern2, filein):
    information1 = []
    information2 = []
    finalinfo = []
    for seq_record in SeqIO.parse(filein, "fasta"):
        result1 = re.search(pattern1,str(seq_record.seq))
        if result1:
            information1.append([seq_record.name ,result1.group(), result1.span(), str(seq_record.description)])
        result2 = re.search(pattern2,str(seq_record.seq))
        if result2:
            information2.append([seq_record.name ,result2.group(), result2.span(), str(seq_record.description)])
    pdpattern1 =pd.DataFrame(information1, columns=['acc','match','start_end','seq'])
    pdpattern2 =pd.DataFrame(information2, columns=['acc','match','start_end','seq'])
    protBoth = pd.merge(pdpattern1, pdpattern2, how='inner', on ='acc')
    return protBoth

intersect(pattern1, pattern2, protFile)

,acc,match_x,start_end_x,seq_x,match_y,start_end_y,seq_y
0,YDR099W,RNLLSVAYKNV,"(42, 53)","YDR099W BMH2 SGDID:S000002506, Chr IV from 653...",YKDSTLIMQLLRDNLTLWTS,"(215, 235)","YDR099W BMH2 SGDID:S000002506, Chr IV from 653..."
1,YER177W,RNLLSVAYKNV,"(42, 53)","YER177W BMH1 SGDID:S000000979, Chr V from 5456...",YKDSTLIMQLLRDNLTLWTS,"(215, 235)","YER177W BMH1 SGDID:S000000979, Chr V from 5456..."


__Exercise 4.__ Parsing and extracting data from a URL:

This is from the tutorial that you should have completed.

When working with files and resources over a network, you will often come across URIs and URLs which can be parsed and worked with directly. Most standard libraries will have classes to parse and construct these kind of identifiers, but if you need to match them in logs or a larger corpus of text, you can use regular expressions to pull out information from their structured format quite easily.

URIs, or Uniform Resource Identifiers, are a representation of a resource that is generally composed of a scheme, host, port (optional), and resource path, respectively highlighted below.

http://regexone.com:80/page

The scheme describes the protocol to communicate with, the host and port describe the source of the resource, and the full path describes the location at the source for the resource.

In the exercise below, try to extract the protocol, host and port of the all the resources listed in this string.

```
ftp://file_server.com:21/top_secret/life_changing_plans.pdf
https://regexone.com/lesson/introduction#section
file://localhost:4040/zip_file
https://s3cur3-server.com:9999/
market://search/angry%20birds
```

You can work interactively here: https://regexone.com/problem/extracting_url_data to find the right regular expression, then use re.finditer to create a dataframe with columns protocol, host and port for each of the matches in the string.

In [24]:
import re

text = """
ftp://file_server.com:21/top_secret/life_changing_plans.pdf
https://regexone.com/lesson/introduction#section
file://localhost:4040/zip_file
https://s3cur3-server.com:9999/
market://search/angry%20birds
"""

#protocol
print("protocol:")
for m in re.finditer('(\w+).//',text):
    print('\t', m.start(), m.group(), m.span())


#host
print('host:')
for m in re.finditer('://([\w\-\.]+)',text):
    print('\t', m.start(), m.group(), m.span())

#port
print('port:')
for m in re.finditer(':(\d+)',text):
    print('\t', m.start(), m.group(), m.span())




protocol:
	 1 ftp:// (1, 7)
	 61 https:// (61, 69)
	 110 file:// (110, 117)
	 141 https:// (141, 149)
	 173 market:// (173, 182)
host:
	 4 ://file_server.com (4, 22)
	 66 ://regexone.com (66, 81)
	 114 ://localhost (114, 126)
	 146 ://s3cur3-server.com (146, 166)
	 179 ://search (179, 188)
port:
	 22 :21 (22, 25)
	 126 :4040 (126, 131)
	 166 :9999 (166, 171)
